In [1]:
import quimb.tensor as qtn
import numpy as np
from weights import ch_weight,sh_weight
from transformations import s_transform
from mps_generators import kraus_mps

use_torch=True


L=20 #use even numbers
J=1
h=1.5
tol=1e-8
delta_t=.08
final_t=3.2
a=33 #0.05 33 #0.1 #32
num_steps=int(final_t/delta_t)

options={}
options['split_opts']={'method':'svd','cutoff':tol,'cutoff_mode':'rel','max_bond':100}
options['tebd_opts']={'tol':tol,'order':4,'progbar':False}
options['comp_opts']={'method':'svd','cutoff':tol,'cutoff_mode':'rel'}



I=np.eye(2)
X=np.array([[0,1],[1,0]])
Z=np.array([[1,0],[0,-1]])

h2=-J*(np.kron(np.kron(Z,I),np.kron(Z,I))-np.kron(np.kron(I,Z),np.kron(I,Z)))
h1=h*(np.kron(X,I)-np.kron(I,X))

ham_tp=qtn.LocalHam1D(L,H2=h2,H1=h1,cyclic=False)
ham_tm=qtn.LocalHam1D(L,H2=-h2,H1=-h1,cyclic=False)
kraus=kraus_mps(Z,int(L/2),L)
weight_func=lambda x: sh_weight(x,a)
sa=s_transform(kraus,weight_func,delta_t,num_steps,ham_tp,ham_tm,use_torch=use_torch**options)


/home/kabel/miniconda3/envs/gibbssampler/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[4,14.7],[6,21.9],[8,33.8],[10,41.3],[12,53.7],[14,62.6],[16,72.2],[18,82.4],[20,84,2],[22,102.6]

[4,7.3],[6,11.7],[8,13.5],[10,19.1],[12,22.8],[14,26.8],[16,30.3],[18,36.3],[20,34.6],[22,39.8]